In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import HuberRegressor

pd.set_option('display.max_columns', None)
#drop_col = ['id','Row#','MaxOfUpperTRange','MinOfUpperTRange','AverageOfUpperTRange','MaxOfLowerTRange','MinOfLowerTRange','AverageOfLowerTRange']
data = pd.read_csv('train.csv').drop(['id','Row#'],axis=1) # ma'lumotni o'qish
data


,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
0,12.5,0.25,0.25,0.25,0.75,69.7,42.1,58.2,50.2,24.3,41.2,16.00,0.26,0.477941,0.423927,34.043022,6079.08526
1,25.0,0.00,0.00,0.00,0.00,86.0,52.0,71.9,62.0,30.0,50.8,3.77,0.06,0.334594,0.354415,27.735098,2946.92602
2,25.0,0.50,0.25,0.75,0.63,86.0,52.0,71.9,62.0,30.0,50.8,34.00,0.56,0.468192,0.417915,34.838815,5323.30034
3,25.0,0.50,0.25,0.38,0.75,69.7,42.1,58.2,50.2,24.3,41.2,24.00,0.39,0.500558,0.427431,35.134955,6157.05484
4,12.5,0.25,0.38,0.50,0.75,77.4,46.8,64.7,55.8,27.0,45.8,24.00,0.39,0.520181,0.464067,37.029180,6992.82314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,25.0,0.50,0.38,0.38,0.50,77.4,46.8,64.7,55.8,27.0,45.8,24.00,0.39,0.489723,0.435764,34.769578,5628.71053
14996,12.5,0.25,0.25,0.25,0.50,94.6,57.2,79.0,68.2,33.0,55.9,16.00,0.26,0.506988,0.458076,36.362837,6331.86722
14997,12.5,0.25,0.38,0.50,0.50,77.4,46.8,64.7,55.8,27.0,45.8,1.00,0.10,0.615612,0.514570,45.718182,8413.65966
14998,12.5,0.25,0.25,0.38,0.75,69.7,42.1,58.2,50.2,24.3,41.2,16.00,0.26,0.551234,0.463896,38.130192,6620.09877


In [ ]:

SEED  = 1
# 1. Seed-to-Fruit Mass Ratio
data['SeedToFruitMassRatio'] = data['seeds'] / (data['fruitmass'] + 1e-5)  # adding small value to avoid division by zero
# 3. Bee Activity Index
data['BeeActivityIndex'] = data['honeybee'] + data['bumbles'] + data['andrena'] + data['osmia']
# 4. Rain Impact Factor
data['RainImpactFactor'] = data['RainingDays'] + data['AverageRainingDays']
# 5. Clone Size Impacted Fruit Mass
data['CloneSizeFruitMass'] = data['clonesize'] * data['fruitmass']


In [ ]:

# Target (label) va features (xususiyatlar) ni ajratamiz
X = data.drop(columns=['yield'])
y = data['yield']

# Ma'lumotlarni o'qitish va test uchun ajratamiz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)




In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso

kf = KFold(n_splits=5,shuffle=True,random_state=SEED)

# Stack uchun qo'shimcha modellarni tanlaymiz
ridge = Ridge(alpha=0.0001,max_iter=1500, random_state=SEED)
lasso = Lasso(alpha=0.03,max_iter=1500, random_state=SEED)
dt = DecisionTreeRegressor(max_depth=6, random_state=SEED)
bc = BaggingRegressor(estimator=dt, n_estimators=400, n_jobs=-1, random_state=SEED)
rf = RandomForestRegressor(n_estimators=150,
                           max_depth=9,
                           criterion='absolute_error')

# Stack modelini kengaytiramiz
stacking_model = StackingRegressor(
    estimators=[
        ('dt',dt),
        ('bc', bc),          # BaggingClassifier
        ('rf', rf),          # RandomForestClassifier
        ('ridge', ridge),    # Ridge
        ('lasso', lasso),    # Lasso
    ],
    final_estimator=HuberRegressor(max_iter=1000),
    cv=kf,
    n_jobs=-1
)

# Pipeline'ni yangilab, PolynomialFeatures va yangi stacking modelni qo'shamiz
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('rfecv', RFECV(estimator=Lasso(max_iter=500), step=1, cv=kf, scoring='neg_mean_absolute_error', n_jobs=-1)),
    ('stacking', stacking_model)
])

# Modelni o'qitish
pipe.fit(X_train, y_train)

# Bashorat qilish va MAE metrikasini hisoblash
#y_pred = pipe.predict(X_test)
print(f"Enhanced Pipeline Model MAE: {mean_absolute_error(y_test, pipe.predict(X_test)):.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.115e+06, tolerance: 2.228e+06
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.120e+06, tolerance: 2.228e+06
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.122e+06, tolerance: 2.228e

Enhanced Pipeline Model MAE: 238.2689


In [ ]:

#Test ma'lumotlarini yuklash
df_test = pd.read_csv('test.csv').drop(['id','Row#'],axis=1)
# 1. Seed-to-Fruit Mass Ratio
df_test['SeedToFruitMassRatio'] = df_test['seeds'] / (df_test['fruitmass'] + 1e-5)  # adding small value to avoid division by zero
# 3. Bee Activity Index
df_test['BeeActivityIndex'] = df_test['honeybee'] + df_test['bumbles'] + df_test['andrena'] + df_test['osmia']
# 4. Rain Impact Factor
df_test['RainImpactFactor'] = df_test['RainingDays'] + df_test['AverageRainingDays']
# 5. Clone Size Impacted Fruit Mass
df_test['CloneSizeFruitMass'] = df_test['clonesize'] * df_test['fruitmass']

# Test to‘plamida ehtimollarni bashorat qilish (Stacking bilan)
y_test_prob_rf = pipe.predict(df_test)
# Bashoratlarni saqlash
subm = pd.read_csv("sample_submission.csv")
subm['yield'] = y_test_prob_rf
subm.to_csv("Haqnazar_submission_colab_best.csv", index=False)